In [1]:
#%env GOOGLE_APPLICATION_CREDENTIALS=/home/pitfox/data/spark-container-dev-f5d53ab2439c.json

env: GOOGLE_APPLICATION_CREDENTIALS=/home/pitfox/data/spark-container-dev-f5d53ab2439c.json


In [2]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/home/pitfox/data/spark-container-dev-f5d53ab2439c.json


In [3]:
!echo $SPARK_HOME

/usr/local/spark-3.1.1-bin-hadoop3.2


In [4]:
!cat $SPARK_HOME/conf/spark-defaults.conf

spark.hadoop.google.cloud.auth.service.account.enable true
spark.hadoop.google.cloud.auth.service.account.json.keyfile /home/pitfox/data/spark-container-dev-f5d53ab2439c.json
spark.hadoop.fs.AbstractFileSystem.gs.impl com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS


In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pyspark.sql.functions as f
import re

In [6]:
spark = SparkSession \
    .builder \
    .appName("Initial file download") \
    .getOrCreate()

In [7]:
category = 'AMAZON_FASHION'

In [8]:
url = f"http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/{category}_5.json.gz"

In [9]:
spark.sparkContext.addFile(url)

In [10]:
df = spark.read.json("file://"+SparkFiles.get("AMAZON_FASHION_5.json.gz"))

In [11]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Size Name:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |    |-- Style:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [12]:
df.show(10)

+----------+-----+-------+--------------------+----------+--------------+------------+--------------------+----------+--------------+--------+----+
|      asin|image|overall|          reviewText|reviewTime|    reviewerID|reviewerName|               style|   summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+----------+--------------+------------+--------------------+----------+--------------+--------+----+
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015| ALJ66O1Y6SLHA|    Tonya B.|{ Blue/Orange, nu...|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015| ALJ66O1Y6SLHA|    Tonya B.|{ Black (37467610...|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015| ALJ66O1Y6SLHA|    Tonya B.|{ Blue/Gray Logo,...|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015| ALJ66O1Y6SLHA|    Tonya B.|{ Blue (37867638-...|Five 

In [13]:
df = df.dropDuplicates() 

In [14]:
df = df.withColumn('review_wordCount', f.size(f.split(f.col('reviewText'), ' ')))

In [15]:
df.registerTempTable("dataframe")

In [16]:
sql_script = f"""select 
          '{category}' as category,
          asin || '-' || reviewerID || row_number() OVER (PARTITION BY asin, reviewerID ORDER BY unixReviewTime asc) as review_ID,
          asin as product_ID,
          reviewerID as reviewer_ID,
          overall as rating_out_of_5,
          summary as review_summary,
          reviewText as review_text,
          review_wordCount as review_word_count,     
          '{url}' as source_url
        from dataframe"""

In [17]:
output = spark.sql(sql_script)

In [18]:
output.show(3)

+--------------+--------------------+----------+--------------+---------------+--------------+--------------------+-----------------+--------------------+
|      category|           review_ID|product_ID|   reviewer_ID|rating_out_of_5|review_summary|         review_text|review_word_count|          source_url|
+--------------+--------------------+----------+--------------+---------------+--------------+--------------------+-----------------+--------------------+
|AMAZON_FASHION|B000YFSR5G-AGZ5OO...|B000YFSR5G| AGZ5OOZVDO194|            5.0|    Five Stars|         Comfortable|                1|http://deepyeti.u...|
|AMAZON_FASHION|B0014F7B98-A2GOOF...|B0014F7B98|A2GOOFFNUI20XI|            5.0|    Five Stars|They fit perfect,...|                5|http://deepyeti.u...|
|AMAZON_FASHION|B0014F7B98-A2RI4I...|B0014F7B98|A2RI4IA5982B8I|            4.0|    Four Stars|Light-weight comf...|                3|http://deepyeti.u...|
+--------------+--------------------+----------+--------------+-------

In [19]:
# Update to your GCS bucket
gcs_bucket = f'{category}'

gcs_filepath = f'gs://amazon_reviews_bucket/{category}'

In [20]:
output.write.partitionBy("product_ID").csv(gcs_filepath)

Py4JJavaError: An error occurred while calling o44.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:469)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:569)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:438)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:415)
	at org.apa

In [21]:
output.coalesce(1).write \
  .mode('overwrite') \
  .csv(gcs_filepath)

Py4JJavaError: An error occurred while calling o48.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:469)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:569)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:438)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:415)
	at org.apa